# This notebook will be used to load a pre-trained model and save the outputs layer by layer

Tensorflow will be used for this endeavor

## sources

[1] VGG16 run: https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/
[2] Layers output: https://stackoverflow.com/questions/41711190/keras-how-to-get-the-output-of-each-layer


Loading imports


In [2]:
# import
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import matplotlib.pyplot as plt
import os 
import numpy as np


Preprocess the data

In [3]:
# load image from path
path = './data/input/mug.jpg'
image = load_img(path, target_size=(224, 224))
# convert raw pixels image to numpy array 
image = img_to_array(image)
#reshape the tensor for VGG16 input
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#call preprocessing for VGG16
image = preprocess_input(image)

Building VGG model


In [4]:
model = VGG16(
    include_top=True,
    weights='imagenet',
    classes=1000,
    classifier_activation='softmax'
)

print(model.summary())

Running model

In [ ]:
# read the output logits from VGG16 on our mug input
logits = model.predict(image)
# convert logits to class labels 
label = decode_predictions(logits)
# get highest confidence label
label = label[0][0]
# print classification 
print('%s (%.2f%%)' % (label[1], label[2]*100))

Capture Layers' output
